In [11]:
import pandas as pd
import os
import numpy as np

def process_stock_data(main_data_path):
    # 고정된 파일 경로
    kospi_data_path = "./Data/Stock/Kospi.csv"
    stock_data_dir = "./Data/Stock"
    result_dir = "./Result"

    # Result 폴더 생성 (존재하지 않으면 생성)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    # 메인 데이터 불러오기
    main_df = pd.read_csv(main_data_path)
    main_df['Predict'] = main_df['Predict'].str.strip()

    # KOSPI 데이터 불러오기
    kospi_df = pd.read_csv(kospi_data_path)
    kospi_df['Date'] = kospi_df['Date'].str.strip()

    # 결과 저장용 데이터프레임 초기화
    result_df = pd.DataFrame(columns=[
        "Date", "Long", "Short", "Long_Short", "Cumulative_Long", 
        "Cumulative_Short", "Cumulative_Long_Short", "Kospi", "Cumulative_Kospi"
    ])

    # 긍정/부정 회사 정보 저장용 데이터프레임 초기화
    detail_df = pd.DataFrame(columns=["Date", "Type", "Company", "Next_Day_Change"])

    # 초기값 (누적 수익률 계산을 위한 초기값)
    cumulative_long = 1
    cumulative_short = 1
    cumulative_long_short = 1
    cumulative_kospi = 1

    for date in main_df['Date'].unique():
        # 해당 날짜의 긍정, 부정 데이터 추출
        date_positive = main_df[(main_df['Date'] == date) & (main_df['Predict'] == '긍정')]
        date_negative = main_df[(main_df['Date'] == date) & (main_df['Predict'] == '부정')]

        # 긍정 및 부정 누적 수익률 계산
        positive_changes = []
        negative_changes = []

        for company in main_df['Company'].unique():
            stock_file = os.path.join(stock_data_dir, f"{company}_Stock.csv")
            
            if os.path.exists(stock_file):
                stock_df = pd.read_csv(stock_file)
                stock_df['Date'] = stock_df['Date'].str.strip()
                
                # 긍정 처리 (Long)
                if company in date_positive['Company'].values:
                    positive_change = stock_df[stock_df['Date'] == date]['Next_Day_Change'].mean()
                    if not pd.isna(positive_change) and not np.isinf(positive_change):
                        positive_changes.append((company, positive_change))
                        detail_df = pd.concat([detail_df, pd.DataFrame({
                            "Date": [date], "Type": ["Long"], "Company": [company], "Next_Day_Change": [positive_change]
                        })])
                
                # 부정 처리 (Short)
                if company in date_negative['Company'].values:
                    negative_change = stock_df[stock_df['Date'] == date]['Next_Day_Change'].mean()
                    if not pd.isna(negative_change) and not np.isinf(negative_change):
                        negative_changes.append((company, negative_change))
                        detail_df = pd.concat([detail_df, pd.DataFrame({
                            "Date": [date], "Type": ["Short"], "Company": [company], "Next_Day_Change": [negative_change]
                        })])

        # 긍정 및 부정 평균 계산 (inf 제외)
        if positive_changes:
            valid_positive_changes = [val for _, val in positive_changes if not np.isinf(val)]
            long_daily = sum(valid_positive_changes) / len(valid_positive_changes) if valid_positive_changes else 0
        else:
            long_daily = 0

        if negative_changes:
            valid_negative_changes = [val for _, val in negative_changes if not np.isinf(val)]
            short_daily = -sum(valid_negative_changes) / len(valid_negative_changes) if valid_negative_changes else 0
        else:
            short_daily = 0

        long_short_daily = (long_daily + short_daily) / 2

        # 누적 수익률 계산
        cumulative_long *= (1 + long_daily / 100)
        cumulative_short *= (1 + short_daily / 100)
        cumulative_long_short *= (1 + long_short_daily / 100)

        # KOSPI 데이터에서 Next_Day_Change 가져오기
        kospi_row = kospi_df[kospi_df['Date'] == date]
        kospi_change = kospi_row['Next_Day_Change'].values[0] if not kospi_row.empty else 0
        kospi_change = 0 if pd.isna(kospi_change) else kospi_change

        # KOSPI 누적 수익률 계산
        cumulative_kospi *= (1 + kospi_change / 100)

        # 결과 저장
        result_df = pd.concat([result_df, pd.DataFrame({
            "Date": [date],
            "Long": [long_daily],
            "Short": [short_daily],
            "Long_Short": [long_short_daily],
            "Cumulative_Long": [cumulative_long],
            "Cumulative_Short": [cumulative_short],
            "Cumulative_Long_Short": [cumulative_long_short],
            "Kospi": [kospi_change],
            "Cumulative_Kospi": [cumulative_kospi]
        })])

    # 날짜별 하위 폴더 생성 및 결과 저장
    result_subdir = os.path.join(result_dir, "Result")
    detail_subdir = os.path.join(result_dir, "Detail")

    os.makedirs(result_subdir, exist_ok=True)
    os.makedirs(detail_subdir, exist_ok=True)

    # 결과 데이터프레임 정렬 및 저장
    result_df = result_df.sort_values(by="Date").reset_index(drop=True)
    detail_df = detail_df.sort_values(by=["Date", "Type", "Company"]).reset_index(drop=True)

    # 결과 파일 저장
    base_filename = os.path.basename(main_data_path).replace(".csv", "")
    output_result_file = os.path.join(result_subdir, f"{base_filename}_Result.csv")
    output_detail_file = os.path.join(detail_subdir, f"{base_filename}_Details.csv")

    result_df.to_csv(output_result_file, index=False, encoding="utf-8-sig")
    detail_df.to_csv(output_detail_file, index=False, encoding="utf-8-sig")

    print(f"결과가 {output_result_file} 및 {output_detail_file}에 저장되었습니다.")
    return result_df, detail_df


In [ ]:
import os
import warnings
import pandas as pd
import numpy as np

# FutureWarning 끄기
warnings.simplefilter(action='ignore', category=FutureWarning)
# Sentiment 디렉터리 경로
sentiment_dir = "./Sentiment"

# Sentiment 폴더 내 모든 CSV 파일 처리
for file_name in os.listdir(sentiment_dir):
    if file_name.endswith(".csv"):  # CSV 파일만 처리
        file_path = os.path.join(sentiment_dir, file_name)  # 파일 경로 생성
        try:
            # CSV 파일 읽기
            df = pd.read_csv(file_path, nrows=1)  # 첫 행만 읽어 구조 확인
            if "Predict" in df.columns:  # Predict 열이 있는 파일만 처리
                print(f"Processing {file_path}...")
                process_stock_data(file_path)  # 함수 호출
            else:
                print(f"Skipping {file_path} (no 'Predict' column).")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

Processing ./Sentiment/gemma2_27b_CoT.csv...
결과가 ./Result/Result/gemma2_27b_CoT_Result.csv 및 ./Result/Detail/gemma2_27b_CoT_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_1_8b_KRFinBERT-BOOTICL.csv...
결과가 ./Result/Result/llama3_1_8b_KRFinBERT-BOOTICL_Result.csv 및 ./Result/Detail/llama3_1_8b_KRFinBERT-BOOTICL_Details.csv에 저장되었습니다.
Processing ./Sentiment/gemma2_27b_KCBERT-BOOTICL.csv...
결과가 ./Result/Result/gemma2_27b_KCBERT-BOOTICL_Result.csv 및 ./Result/Detail/gemma2_27b_KCBERT-BOOTICL_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_2_3b_NOCoT.csv...
결과가 ./Result/Result/llama3_2_3b_NOCoT_Result.csv 및 ./Result/Detail/llama3_2_3b_NOCoT_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_2_3b_KBAlbert-ICL.csv...
결과가 ./Result/Result/llama3_2_3b_KBAlbert-ICL_Result.csv 및 ./Result/Detail/llama3_2_3b_KBAlbert-ICL_Details.csv에 저장되었습니다.
Processing ./Sentiment/KCBERT.csv...
결과가 ./Result/Result/KCBERT_Result.csv 및 ./Result/Detail/KCBERT_Details.csv에 저장되었습니다.
Processing ./Sentiment/llama3_2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def save_cumulative_returns_chart(csv_file_path, save_folder="./Result/Chart"):
    # 데이터 로드
    df = pd.read_csv(csv_file_path)
    
    # Date 열을 datetime 형식으로 변환
    df['Date'] = pd.to_datetime(df['Date'])

    # Chart 저장 폴더 생성 (존재하지 않으면 생성)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # 저장 파일명 생성
    base_filename = os.path.basename(csv_file_path).replace("_Result.csv", "")
    save_path = os.path.join(save_folder, f"{base_filename}_Chart.png")

    # 그래프 생성
    plt.figure(figsize=(12, 6))

    # 각 누적 수익률을 그래프로 표시
    plt.plot(df['Date'], df['Cumulative_Long'], label='Cumulative Long', marker='o', linestyle='-')
    plt.plot(df['Date'], df['Cumulative_Short'], label='Cumulative Short', marker='o', linestyle='--')
    plt.plot(df['Date'], df['Cumulative_Long_Short'], label='Cumulative Long-Short', marker='o', linestyle='-.')
    plt.plot(df['Date'], df['Cumulative_Kospi'], label='Cumulative Kospi', marker='o', linestyle=':')

    # 그래프 설정
    plt.title(base_filename, fontsize=16)  # 파일명을 제목으로 설정
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("Cumulative Return", fontsize=14)
    plt.grid(True)
    plt.legend(loc='upper left', fontsize=12)

    # X축 월별 표시
    plt.xticks(df['Date'][::30], df['Date'][::30].dt.strftime('%Y-%m'), rotation=45)

    plt.tight_layout()

    # 그래프 저장
    plt.savefig(save_path, format='png')
    plt.close()

    print(f"차트가 {save_path}에 저장되었습니다.")


In [ ]:
import os

# Result 디렉터리 내 모든 CSV 파일 처리
result_dir = "./Result/Result"

for file_name in os.listdir(result_dir):
    if file_name.endswith(".csv"):  # CSV 파일만 처리
        csv_file_path = os.path.join(result_dir, file_name)
        try:
            print(f"Processing {csv_file_path}...")
            save_cumulative_returns_chart(csv_file_path)  # 함수 호출
        except Exception as e:
            print(f"Error processing {csv_file_path}: {e}")

In [ ]:
import os
import pandas as pd

# Result/Result 폴더 경로
result_root_folder = "./Result/Result"
output_summary_path = os.path.join("./Result", "Summary_Analysis.csv")

# 전체 결과 저장용 리스트
final_results_list = []

# Kospi 결과 추가 여부를 추적
kospi_added = False

# 모든 파일 순회
for file_name in os.listdir(result_root_folder):
    if file_name.endswith("_Result.csv"):  # _Result.csv 파일만 처리
        file_path = os.path.join(result_root_folder, file_name)
        data = pd.read_csv(file_path)

        # 전략별 분석
        strategies = ['Long', 'Short', 'Long_Short']

        # Kospi 데이터 분석 (한 번만 추가)
        if 'Kospi' in data.columns and not kospi_added:
            kospi_returns = data['Kospi'].dropna()
            if not kospi_returns.empty:
                kospi_mean_return = kospi_returns.mean() / 100
                kospi_std_dev_return = kospi_returns.std() / 100
                kospi_sharpe_ratio = (kospi_mean_return / kospi_std_dev_return) * (252 ** 0.5) if kospi_std_dev_return != 0 else 0
                kospi_cumulative_values = (1 + kospi_returns / 100).cumprod()
                kospi_peak = kospi_cumulative_values.cummax()
                kospi_drawdown = (kospi_cumulative_values - kospi_peak) / kospi_peak
                kospi_mdd = kospi_drawdown.min() * 100  # MDD를 %로 변환
                kospi_final_return = (kospi_cumulative_values.iloc[-1] - 1) * 100  # Final_Return을 %로 변환

                final_results_list.append({
                    'Path': "Kospi",
                    'Strategy': "Kospi",
                    'Mean_Return': kospi_mean_return * 100,  # % 단위로 변환
                    'Std_Dev_Return': kospi_std_dev_return * 100,  # % 단위로 변환
                    'Sharpe_Ratio': kospi_sharpe_ratio,
                    'MDD': kospi_mdd,
                    'Final_Return': kospi_final_return
                })

                kospi_added = True  # Kospi 결과 추가 완료

        # 각 전략 분석
        for strategy in strategies:
            if strategy in data.columns:
                returns = data[strategy].dropna()
                if not returns.empty:
                    mean_return = returns.mean() / 100
                    std_dev_return = returns.std() / 100
                    sharpe_ratio = (mean_return / std_dev_return) * (252 ** 0.5) if std_dev_return != 0 else 0

                    # 누적 수익률 계산 (1 + 수익률로 누적 곱 계산)
                    cumulative_values = (1 + returns / 100).cumprod()
                    peak = cumulative_values.cummax()
                    drawdown = (cumulative_values - peak) / peak
                    mdd = drawdown.min() * 100  # MDD를 %로 변환
                    final_return = (cumulative_values.iloc[-1] - 1) * 100  # Final_Return을 %로 변환

                    final_results_list.append({
                        'Path': file_path,
                        'Strategy': strategy,
                        'Mean_Return': mean_return * 100,  # % 단위로 변환
                        'Std_Dev_Return': std_dev_return * 100,  # % 단위로 변환
                        'Sharpe_Ratio': sharpe_ratio,
                        'MDD': mdd,
                        'Final_Return': final_return
                    })

# 결과 저장
final_results = pd.DataFrame(final_results_list)
final_results.to_csv(output_summary_path, index=False, encoding='utf-8-sig')

print(f"결과 요약 파일이 저장되었습니다: {output_summary_path}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Summary_Analysis.csv 불러오기
df = pd.read_csv("./Result/Summary_Analysis.csv")

# Sharpe_Ratio 기준으로 상위 5개 추출
top_5_sharpe = df.nlargest(5, 'Sharpe_Ratio')

# 그래프 그리기
plt.figure(figsize=(12, 6))

# 나스닥 데이터 여부 확인
nasdaq_data = None

for index, row in top_5_sharpe.iterrows():
    # Path와 Strategy 가져오기
    path = row['Path']
    strategy = row['Strategy']
    
    # CSV 파일 읽기
    data = pd.read_csv(path)
    data['Date'] = pd.to_datetime(data['Date'])
    data = data.sort_values('Date')  # 날짜 정렬
    
    # 파일명 추출 및 형식 지정
    file_name = os.path.basename(path).replace("_Result.csv", "")
    label = f"{file_name}-{strategy}"  # 범례 형식: 파일명-전략명

    # 누적 수익률 계산
    cumulative_column = f"Cumulative_{strategy}" if f"Cumulative_{strategy}" in data.columns else strategy
    if cumulative_column in data.columns:
        plt.plot(data['Date'], data[cumulative_column], label=label)
    
    # 나스닥 데이터 추가
    if 'Kospi' in data.columns and nasdaq_data is None:
        nasdaq_data = data[['Date', 'Cumulative_Kospi']].dropna()

# 나스닥 그래프 추가
if nasdaq_data is not None:
    plt.plot(nasdaq_data['Date'], nasdaq_data['Cumulative_Kospi'], label="Kospi", linestyle='--', color='black')

# 그래프 설정
plt.title("Top 5 Strategies and Kospi Cumulative Returns", fontsize=16)
plt.xlabel("Date", fontsize=14)
plt.ylabel("Cumulative Return", fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True)
plt.tight_layout()

# 그래프 출력
plt.show()
